# Run Datasets

## MacLaren
```
submit_subjects \
  --upload_metadata \
  --save_details \
  --stagger \
  -q reTHINQ-c5-spot \
  -t 1.0.0-rc.11 \
  -I s3://cmet-scratch/maclaren-cmeds/ \
  -o s3://cmet-scratch/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/
```

## DefNef
```
submit_subjects \
  --upload_metadata \
  --save_details \
  --stagger \
  -q reTHINQ-c5-spot \
  -t 1.0.0-rc.11 \
  -I s3://cmet-testsets/DecNefTS/ \
  -o s3://cmet-scratch/20200615-DecNefTS-1.0.0-rc.11/
```

# Copy Data Locally

## Maclaren
```
mkdir -p /home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4
cd /home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*subject_info.json" \
  --include "*.pdf" \
  s3://cmet-scratch/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/maclaren-cmeds/ .
find . -type d -name 'cache' -exec rm -rf {} \;
```

## DecNef
```
mkdir -p /home/paul/cmet/data/20200615-DecNefTS-1.0.0-rc.11
cd /home/paul/cmet/data/20200615-DecNefTS-1.0.0-rc.11
aws s3 cp s3://cmet-testsets/DecNefTS/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*subject_info.json" \
  --include "*.pdf" \
  --include "*.log" \
  s3://cmet-scratch/20200615-DecNefTS-1.0.0-rc.11/DecNefTS/ .
rm -f ./sub-058/cache/59d231cda1504d2d/rethinq/subject_info.json
```

In [1]:
import json
import os
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Local DecNef Vars
decnef_dir = '/home/paul/cmet/data/20200615-DecNefTS-1.0.0-rc.11/'
decnef_tsv = '/home/paul/cmet/data/20200615-DecNefTS-1.0.0-rc.11/demographics.tsv'
# Local MacLaren Vars
maclaren_dir = '/home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/'
maclaren_tsv = '/home/paul/cmet/data/20200609-mclaren-1.0.0-rc.11-42-g8d976b0--take4/demographics.tsv'

In [3]:
# Helper functions

def find_json_files(filepath, pattern="*.json"):
    """
    Given:
    - a directory to search

    Produce:
    - a list of all .json files anywhere in the directory path
    - equivalient to the shell command `find /filepath -name '*.json'`
    """
    filelist = []
    for dName, sdName, fList in os.walk(filepath):
        for fileName in fList:
            if fnmatch.fnmatch(fileName, pattern):
                filelist.append(os.path.join(dName, fileName))
    return filelist

def load_json_file(filename):
    """
    Given:
    - a json file

    Produce:
    - dictionary
    """
    with open(filename) as infile:
        json_data = json.load(infile)
    return json_data

def load_dataset(jsonpath, demofile, drop_subjects=None):
    json_files = find_json_files(jsonpath)
    subject_data = {}
    vol_data = {}
    norm_data = {}
    subject_list = []
    for file in json_files:
        subname = os.path.basename(os.path.dirname(file))
        subject_data[subname] = load_json_file(file)
        if 'normative' in subject_data[subname] and 'volume' in subject_data[subname]['normative']:
            # Subject was processed without error
            vol_data[subname] = subject_data[subname]['measurements']['volume_percent_icv']
            norm_data[subname] = {}
            for vol in subject_data[subname]['normative']['volume']:
                norm_data[subname][vol] = subject_data[subname]['normative']['volume'][vol]['percentiles']['percentile']        
            subject_list.append(subname)
        else:
            # Subject was processed with error
            if drop_subjects is None:
                drop_subjects = []
            drop_subjects.append(subname)
            print('Ignoring Subject (did it error out?)', subname)

    demo_dataf = pd.read_csv(demofile, sep='\t', index_col='subject_id')
    vol_temp_df = pd.DataFrame.from_dict(data=vol_data, orient='index')
    norm_temp_df = pd.DataFrame.from_dict(data=norm_data, orient='index')
    if drop_subjects is not None:
        print('Dropping the following subejcts:', drop_subjects)
        demo_dataf.drop(drop_subjects, errors='ignore')
        vol_temp_df.drop(drop_subjects, errors='ignore')
        norm_temp_df.drop(drop_subjects, errors='ignore')
    # for convience, concatenate demographics info onto vol and norm dataframes
    vol_dataf = pd.concat([demo_dataf,vol_temp_df],axis=1)
    norm_dataf = pd.concat([demo_dataf,norm_temp_df],axis=1)

    return vol_dataf, norm_dataf


## Compute Coefficients of Variation

Calculates:

  1) The total coefficient of variation, $CV_t$
  
  2) Intra-session coefficient of variation, $CV_s$ 2 different ways
  
### Computing $CV_t$

For a given subject, $CV_t$ is computed across all measurements from all sessions.  According to [1]:

$CV_t = 100\frac{\sigma_t}{\bar{x}}$

Where $\sigma_t$ is the total standard deviation, computed across all all measurements from all sessions.

$CV_t$ across subjects are RMS averaged together (arithmetic mean is not valid)

### Computing $CV_s$

For a given subject, $m$ is the number of sessions and $n_j$ is the number of repeated measurements in session $j$

#### The MacLaren Method

The MacLaren method[1] works if all sessions have 2 timepoints (all $n_j = 2$) . The Intra-session 
coefficient of variation $CV_s$ is computed for each subject and pooled across subjects via RMS average.
      
$CV_s = 100\frac{\sigma_s}{\bar{x}}$

$\sigma_s = \sqrt{\frac{\sum{(x_i'-x_i'')^2}}{2m}}$     
     
Where $x_i'$ and $x_i''$ are the $i^{th}$ paired measurements over the $m$ pairs

#### The Generalized Glüer method

Using the generalized method in Glüer[2] (eqs 6, 5 and 7 in [2] but using MacLaren notation).  As in MacLaren, the Intra-session coefficient of variation $CV_s$ is computed for each subject and pooled together via RMS average
  
$CV_s = (\sigma_s / \sum_{j=1}^{m}{\bar{x}_j} / m)$  

$\sigma_s = \sqrt{\sum_{j=1}^{m} \sum_{i=1}^{n_j} \frac{(x_{ij}-\bar{x}_j)^2}{df}}$

$df = \sum_{j=1}^m{(N_j-1)}$ 

#### Is it ok to pool $CV_s$ together with simple RMS if each one has a different number of repeated measures?

[1]: Maclaren, Julian, et al. "Reliability of brain volume measurements: a test-retest dataset." Scientific data 1.1 (2014): 1-9. [pdf](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.785.3729&rep=rep1&type=pdf)

[2]: Glüer, C-C., et al. "Accurate assessment of precision errors: how to measure the reproducibility of bone densitometry techniques." Osteoporosis international 5.4 (1995): 262-270. [pdf](https://sci-hub.tw/10.1007/bf01774016)

In [95]:
def calc_cvs(df, subject_list, session_list, subject_row, session_row, structs_of_interest, method='gluer'):
    """
    Given:
        - a dataframe; as returned by load_dataset() (`df`)
        - a list of subjects to iterate over (`subject_list`)
        - a list of sessions to iterate over (assummes all subjects have all sessions) (`session_list`)
        - the row in the dataset's tsv that denotes subject_id (`subject_row`)
        - the row in the dataset's tsv that denotes session_id (`session_row`)
        - the rows in the dataframe over which to compute (`structs_of_interest`)
        - method to employ to compute intra-session CoV ('maclaren' or 'gluer'; default='gluer')
    Produce:
        - a dataframe with the total coefficient of variation (CoV) for each element in `structs_of_interest` 
          (`total_cvs_df`)
        - a datafram with the intra-session coefficient of variation (CoV) for each element in `structs_of_interest`
          session_cvs_df  
    See: 
        - [1] Maclaren, Julian, et al. "Reliability of brain volume measurements: a test-retest dataset." 
          Scientific data 1.1 (2014): 1-9.
        - [2]: Glüer, C-C., et al. "Accurate assessment of precision errors: how to measure the 
          reproducibility of bone densitometry techniques." Osteoporosis international 5.4 (1995): 262-270.
    """

    # Holds the intra-session CoV for each subject/struct
    # Will eventually be a numpy array
    subject_session_cvs = None
    # Holds the total CoV for each subject/struct
    # Will eventually be a numpy array
    subject_total_cvs = None
    
    for subject in subject_list:
        # Select by subject; make a numpy array
        subject_df = df[df[subject_row]==subject]
        subject_level_vals = subject_df.loc[:,structs_of_interest].to_numpy()

        # To track the summation in $\sigma_s = \sqrt{\frac{\sum{(x_i'-x_i'')^2}}{2m}}$ in [1]
        # Will eventually be a numpy array
        subject_sum = None
        
        # same m used to compute sigma_s in [1]
        m = 0

        # Calculate total CoV for this subject
        total_cvs = 100 * np.std(subject_level_vals,axis=0)/np.mean(subject_level_vals,axis=0)
        if subject_total_cvs is None:
            subject_total_cvs = total_cvs
        else:
            subject_total_cvs = np.stack((subject_total_cvs,total_cvs))

        # Compute `subject_session_cvs` according to [1] or eq's 5 and 6 in [2]
        if (method == 'maclaren'):
            # Compute `session_cvs` a la Maclaren
            for session in session_list:
                session_df = subject_df[subject_df[session_row]==session]
                session_level_vals = session_df.loc[:,structs_of_interest].to_numpy()
                vals_has_a_nan = np.isnan(np.sum(session_level_vals))
                if vals_has_a_nan:
                    # At least one of the pairs did not get processed properly; skip
                    # this is fragile and probably needs more work
                    continue
                m += 1
                diff_squared = np.square(np.diff(session_level_vals,axis=0).flatten())
                if subject_sum is None:
                    subject_sum = diff_squared
                else:
                    subject_sum += diff_squared
            sigma_s = np.sqrt(np.divide(subject_sum,2*m))
            session_cvs = 100 * sigma_s / np.mean(subject_level_vals,axis=0)
                
        elif (method=='gluer'):
            # Compute `session_cvs` a la Gluer
            
            m = len(session_list)
            # Get the number of repeated measures in each session
            n_meas = []
            # counter for doube summation term of eq 6 in [2]
            std_ctr_div_df = None
            # counter for the summation term of eq 5 in [2]
            x_j_over_m = None
            
            # Find the number of repeated measurements in each session and compute df (eq 7 in [2])            
            for session in session_list:
                n_meas.append(subject_df[subject_df[session_row]==session].shape[0])
            df = np.sum(np.subtract(n_meas,1))

            # Compute sigma_s (eq 6 in [2])
            for session in session_list:
                session_df = subject_df[subject_df[session_row]==session]
                session_level_vals = session_df.loc[:,structs_of_interest].to_numpy()
                if std_ctr_div_df is None:
                    std_ctr_div_df = np.sum(np.square(np.mean(session_level_vals,axis=0) - session_level_vals)/df,axis=0)
                else:
                    std_ctr_div_df += np.sum(np.square(np.mean(session_level_vals,axis=0) - session_level_vals)/df,axis=0)
                if x_j_over_m is None:
                    x_j_over_m = np.mean(session_level_vals,axis=0)/m
                else:
                    x_j_over_m += np.mean(session_level_vals,axis=0)/m                    
            sigma_s = np.sqrt(std_ctr_div_df)
            session_cvs = 100 * (sigma_s / x_j_over_m)            
        else:
            print('Balls')

        if subject_session_cvs is None:
            subject_session_cvs = session_cvs
        else:
            subject_session_cvs = np.stack((subject_session_cvs,session_cvs))            

    # Now, take the mean of the coefficients of variation for each subject, and RMS average them together
    session_cvs = np.sqrt(np.mean(np.square(subject_session_cvs),axis=0))
    total_cvs = np.sqrt(np.mean(np.square(subject_total_cvs),axis=0))

    # Stuff results back into a dataframe
    session_cvs_df = pd.DataFrame(np.array(session_cvs,ndmin=2), columns = structs_of_interest)
    total_cvs_df = pd.DataFrame(np.array(total_cvs,ndmin=2), columns = structs_of_interest)

    return total_cvs_df, session_cvs_df

In [15]:
# Load DecNef data into dataframes
decnef_vol_df, decnef_norm_df = load_dataset(decnef_dir, decnef_tsv, drop_subjects=['sub-143'])

Ignoring Subject (did it error out?) sub-058
Dropping the following subejcts: ['sub-143', 'sub-058']


In [5]:
# Load MacLaren data into dataframes
maclaren_vol_df, maclaren_norm_df = load_dataset(maclaren_dir, maclaren_tsv)

Ignoring Subject (did it error out?) sub-01_run-39
Ignoring Subject (did it error out?) sub-01_run-02
Ignoring Subject (did it error out?) sub-01_run-09
Ignoring Subject (did it error out?) sub-01_run-08
Ignoring Subject (did it error out?) sub-01_run-24
Ignoring Subject (did it error out?) sub-01_run-33
Ignoring Subject (did it error out?) sub-01_run-13
Ignoring Subject (did it error out?) sub-01_run-16
Ignoring Subject (did it error out?) sub-01_run-14
Ignoring Subject (did it error out?) sub-01_run-32
Ignoring Subject (did it error out?) sub-01_run-01
Ignoring Subject (did it error out?) sub-01_run-36
Ignoring Subject (did it error out?) sub-01_run-06
Ignoring Subject (did it error out?) sub-01_run-26
Ignoring Subject (did it error out?) sub-01_run-27
Ignoring Subject (did it error out?) sub-01_run-40
Ignoring Subject (did it error out?) sub-01_run-03
Ignoring Subject (did it error out?) sub-01_run-19
Ignoring Subject (did it error out?) sub-01_run-18
Ignoring Subject (did it error 

In [17]:
structs_of_interest = [
    'BrainSegVolNotVentSurf',
    'TotalGrayVol',
    'White-Matter',
    'lh_cortex_volume',
    'lh_frontal_volume',
    'lh_parietal_volume',
    'lh_occipital_volume',
    'lh_temporal_volume',
    'Left-White-Matter',
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Left-Thalamus',
    'Left-Cerebellum',
    ]

In [ ]:
structs_of_interest = [
    'BrainSegVolNotVentSurf',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Cerebellum',
    'Left-Hippocampus',
    'Left-Lateral-Ventricle',
    'Left-Putamen',
    'Left-Thalamus',
    'Left-White-Matter',
    'Right-Amygdala',
    'Right-Caudate',
    'Right-Cerebellum',
    'Right-Hippocampus',
    'Right-Lateral-Ventricle',
    'Right-Putamen',
    'Right-Thalamus',
    'Right-White-Matter',
    'TotalGrayVol',
    'White-Matter',
    'lh_cortex_volume',
    'lh_frontal_volume',
    'lh_occipital_volume',
    'lh_parietal_volume',
    'lh_temporal_volume',
    'rh_cortex_volume',
    'rh_frontal_volume',
    'rh_occipital_volume',
    'rh_parietal_volume',
    'rh_temporal_volume',
    'sbTIV'
    ]

In [29]:
structs_of_interest = [
    'BrainSegVolNotVentSurf',
    'TotalGrayVol',
    'White-Matter',
    'lh_cortex_volume',
    'lh_frontal_volume',
    'lh_parietal_volume',
    'lh_occipital_volume',
    'lh_temporal_volume',
    'Left-White-Matter',
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Left-Thalamus',
    'Left-Cerebellum',
    ]
session_row='session'
subject_row='subject_num'
#subject_list= [1,2]
#session_list= [1]
session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
subject_list= [2,3]

total_cvs_df, session_cvs_df = calc_cvs(maclaren_vol_df,subject_list,session_list,subject_row,session_row,structs_of_interest,method='maclaren')

In [26]:
maclaren_norm_df.loc[:,structs_of_interest]

maclaren_norm_df.columns.values

array(['age', 'sex', 'manufacturer', 'field_strength', 'diagnosis',
       'file_type', 'scan_time', 'scan_date', 'subject_num', 'session',
       'source', 'BrainSegVolNotVentSurf', 'Left-Amygdala',
       'Left-Caudate', 'Left-Cerebellum', 'Left-Hippocampus',
       'Left-Lateral-Ventricle', 'Left-Putamen', 'Left-Thalamus',
       'Left-White-Matter', 'Right-Amygdala', 'Right-Caudate',
       'Right-Cerebellum', 'Right-Hippocampus', 'Right-Lateral-Ventricle',
       'Right-Putamen', 'Right-Thalamus', 'Right-White-Matter',
       'TotalGrayVol', 'White-Matter', 'lh_cortex_volume',
       'lh_frontal_volume', 'lh_occipital_volume', 'lh_parietal_volume',
       'lh_temporal_volume', 'rh_cortex_volume', 'rh_frontal_volume',
       'rh_occipital_volume', 'rh_parietal_volume', 'rh_temporal_volume',
       'sbTIV'], dtype=object)

In [27]:
maclaren_vol_df.columns.values

array(['age', 'sex', 'manufacturer', 'field_strength', 'diagnosis',
       'file_type', 'scan_time', 'scan_date', 'subject_num', 'session',
       'source', '3rd-Ventricle', '4th-Ventricle', '5th-Ventricle',
       'Brain-Stem', 'BrainSegVol', 'BrainSegVol-to-eTIV',
       'BrainSegVolNotVent', 'BrainSegVolNotVentSurf', 'CC_Anterior',
       'CC_Central', 'CC_Mid_Anterior', 'CC_Mid_Posterior',
       'CC_Posterior', 'CSF', 'CerebralWhiteMatterVol', 'CortexVol',
       'EstimatedTotalIntraCranialVol', 'Left-Accumbens-area',
       'Left-Amygdala', 'Left-Caudate', 'Left-Cerebellum',
       'Left-Cerebellum-Cortex', 'Left-Cerebellum-White-Matter',
       'Left-Hippocampus', 'Left-Inf-Lat-Vent', 'Left-Lateral-Ventricle',
       'Left-Pallidum', 'Left-Putamen', 'Left-Thalamus', 'Left-VentralDC',
       'Left-WM-hypointensities', 'Left-White-Matter',
       'Left-choroid-plexus', 'Left-non-WM-hypointensities',
       'Left-vessel', 'MaskVol', 'MaskVol-to-eTIV', 'Optic-Chiasm',
       'Right-

In [13]:
session_cvs_df

,BrainSegVolNotVentSurf,Left-Amygdala,Left-Caudate,Left-Cerebellum,Left-Hippocampus,Left-Lateral-Ventricle,Left-Putamen,Left-Thalamus,Left-White-Matter,Right-Amygdala,...,lh_frontal_volume,lh_occipital_volume,lh_parietal_volume,lh_temporal_volume,rh_cortex_volume,rh_frontal_volume,rh_occipital_volume,rh_parietal_volume,rh_temporal_volume,sbTIV
0,5.078402,26.380076,11.134683,5.28008,21.163843,3.47318,15.392453,16.347158,10.57611,15.917936,...,3.020233,7.469712,19.67608,17.710075,6.21479,7.945281,8.171012,11.545097,4.877856,1.773898


In [8]:
total_cvs_df

,BrainSegVolNotVentSurf,Left-Amygdala,Left-Caudate,Left-Cerebellum,Left-Hippocampus,Left-Lateral-Ventricle,Left-Putamen,Left-Thalamus,Left-White-Matter,Right-Amygdala,...,lh_frontal_volume,lh_occipital_volume,lh_parietal_volume,lh_temporal_volume,rh_cortex_volume,rh_frontal_volume,rh_occipital_volume,rh_parietal_volume,rh_temporal_volume,sbTIV
0,7.308646,26.006592,12.772852,11.324928,16.788254,5.110548,21.915473,20.446066,13.538717,16.491264,...,3.993229,9.235421,29.101389,24.089442,7.136662,9.597228,9.123576,11.204808,5.44443,2.161402


# -------------------------------------

# Generate Figs

In [55]:
filestem = 'figs/decnef-vol--'
for struct in structs_of_interest:
    fig, ax = plt.subplots(dpi=400)
    fig.set_size_inches(8, 4)
    p = sns.stripplot(
        x='Subject',
        y=struct,
        hue='manufacturer',
        data=decnef_vol_df
    )
    plt.savefig(filestem+struct+".png")
    plt.close()

In [58]:
filestem = 'figs/decnef-norm--'
for struct in structs_of_interest:
    fig, ax = plt.subplots(dpi=400)
    fig.set_size_inches(8, 4)
    p = sns.stripplot(
        x='Subject',
        y=struct,
        hue='manufacturer',
        data=decnef_norm_df
    )
    plt.savefig(filestem+struct+".png")
    plt.close()

In [62]:
filestem = 'figs/maclaren-vol--'
for struct in structs_of_interest:
    if struct == 'eTIV':
        continue
    fig, ax = plt.subplots(dpi=400)
    fig.set_size_inches(8, 4)
    p = sns.stripplot(
        x='subject_num',
        y=struct,
        data=maclaren_vol_df
    )
    plt.savefig(filestem+struct+".png")
    plt.close()

In [64]:
filestem = 'figs/maclaren-norm--'
for struct in structs_of_interest:
    if struct == 'eTIV':
        continue
    fig, ax = plt.subplots(dpi=400)
    fig.set_size_inches(8, 4)
    p = sns.stripplot(
        x='subject_num',
        y=struct,
        data=maclaren_norm_df
    )
    plt.savefig(filestem+struct+".png")
    plt.close()


In [30]:
maclaren_vol_df

,age,sex,manufacturer,field_strength,diagnosis,file_type,scan_time,scan_date,subject_num,session,...,rh_precuneus_volume,rh_rostralanteriorcingulate_volume,rh_rostralmiddlefrontal_volume,rh_superiorfrontal_volume,rh_superiorparietal_volume,rh_superiortemporal_volume,rh_supramarginal_volume,rh_temporal_volume,rh_temporalpole_volume,rh_transversetemporal_volume
sub-01_run-01,26,M,GE,3,HC,nifti,809,19700101,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01_run-02,26,M,GE,3,HC,nifti,824,19700101,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01_run-03,26,M,GE,3,HC,nifti,818,19700102,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01_run-04,26,M,GE,3,HC,nifti,827,19700102,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-01_run-05,26,M,GE,3,HC,nifti,1413,19700103,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-03_run-36,30,F,GE,3,HC,nifti,850,19700129,3,18,...,0.594424,0.110256,1.167628,1.581370,0.859348,0.795972,0.761006,3.899355,0.155021,0.066689
sub-03_run-37,30,F,GE,3,HC,nifti,839,19700130,3,19,...,0.606583,0.113840,1.175713,1.627618,0.858668,0.807179,0.766763,3.875431,0.166528,0.065737
sub-03_run-38,30,F,GE,3,HC,nifti,848,19700130,3,19,...,0.598375,0.108160,1.176148,1.584942,0.862814,0.800726,0.762344,3.877407,0.155855,0.066745
sub-03_run-39,30,F,GE,3,HC,nifti,1507,19700131,3,20,...,0.596858,0.106771,1.158338,1.598881,0.863750,0.812444,0.764238,3.890962,0.153919,0.064344


In [33]:
n_meas = []
n_meas.append(2)
n_meas.append(3)


In [39]:
df = maclaren_vol_df
structs_of_interest = [
    'BrainSegVolNotVentSurf',
    'TotalGrayVol',
    'White-Matter',
    'lh_cortex_volume',
    'lh_frontal_volume',
    'lh_parietal_volume',
    'lh_occipital_volume',
    'lh_temporal_volume',
    'Left-White-Matter',
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Left-Thalamus',
    'Left-Cerebellum',
    ]
session_row='session'
subject_row='subject_num'
#subject_list= [1,2]
#session_list= [1]
session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
subject_list= [2,3]
session = 1
subject = 2

subject_df = df[df[subject_row]==subject]
subject_level_vals = subject_df.loc[:,structs_of_interest].to_numpy()        
session_df = subject_df[subject_df[session_row]==session]
session_level_vals = session_df.loc[:,structs_of_interest].to_numpy()

In [51]:
session_level_vals

array([[73.47411445, 40.6042195 , 31.19027745, 17.17459809,  6.33992394,
         3.73796566,  1.73787903,  4.1568703 , 15.52008164,  0.63621335,
         0.26803991,  0.09813406,  0.23463336,  0.37473786,  0.39975278,
         4.78692527],
       [73.45975974, 40.66950996, 31.12348321, 17.28193491,  6.38060807,
         3.78767573,  1.74216002,  4.18166332, 15.54266282,  0.63199001,
         0.26503837,  0.09800533,  0.23155066,  0.37363689,  0.40558847,
         4.79227649]])

In [73]:
mean = 
mean_diff = 
(mean_diff + session_level_vals) - mean

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [78]:
df=10


array([[5.15144321e-06, 1.06571097e-04, 1.11536754e-04, 2.88029818e-04,
        4.13799617e-05, 6.17772913e-05, 4.58172484e-07, 1.53673483e-05,
        1.27477429e-05, 4.45914785e-07, 2.25230224e-07, 4.14239616e-10,
        2.37575568e-07, 3.03033063e-08, 8.51381372e-07, 7.15888219e-07],
       [5.15144321e-06, 1.06571097e-04, 1.11536754e-04, 2.88029818e-04,
        4.13799617e-05, 6.17772913e-05, 4.58172484e-07, 1.53673483e-05,
        1.27477429e-05, 4.45914785e-07, 2.25230224e-07, 4.14239616e-10,
        2.37575568e-07, 3.03033063e-08, 8.51381372e-07, 7.15888219e-07]])

In [61]:
session_level_vals - np.mean(session_level_vals,axis=0,keepdims=True)

array([[ 7.17735551e-03, -3.26452289e-02,  3.33971187e-02,
        -5.36684096e-02, -2.03420652e-02, -2.48550380e-02,
        -2.14049640e-03, -1.23965109e-02, -1.12905903e-02,
         2.11166945e-03,  1.50076722e-03,  6.43614493e-05,
         1.54134866e-03,  5.50484389e-04, -2.91784402e-03,
        -2.67560875e-03],
       [-7.17735551e-03,  3.26452289e-02, -3.33971187e-02,
         5.36684096e-02,  2.03420652e-02,  2.48550380e-02,
         2.14049640e-03,  1.23965109e-02,  1.12905903e-02,
        -2.11166945e-03, -1.50076722e-03, -6.43614493e-05,
        -1.54134866e-03, -5.50484389e-04,  2.91784402e-03,
         2.67560875e-03]])

In [55]:
np.square(np.mean(session_level_vals,axis=0) - session_level_vals)/df

array([[ 7.17735551e-03, -3.26452289e-02,  3.33971187e-02,
        -5.36684096e-02, -2.03420652e-02, -2.48550380e-02,
        -2.14049640e-03, -1.23965109e-02, -1.12905903e-02,
         2.11166945e-03,  1.50076722e-03,  6.43614493e-05,
         1.54134866e-03,  5.50484389e-04, -2.91784402e-03,
        -2.67560875e-03],
       [-7.17735551e-03,  3.26452289e-02, -3.33971187e-02,
         5.36684096e-02,  2.03420652e-02,  2.48550380e-02,
         2.14049640e-03,  1.23965109e-02,  1.12905903e-02,
        -2.11166945e-03, -1.50076722e-03, -6.43614493e-05,
        -1.54134866e-03, -5.50484389e-04,  2.91784402e-03,
         2.67560875e-03]])

In [80]:
foo = np.square(np.mean(session_level_vals,axis=0) - session_level_vals)/df
foo

array([[5.15144321e-06, 1.06571097e-04, 1.11536754e-04, 2.88029818e-04,
        4.13799617e-05, 6.17772913e-05, 4.58172484e-07, 1.53673483e-05,
        1.27477429e-05, 4.45914785e-07, 2.25230224e-07, 4.14239616e-10,
        2.37575568e-07, 3.03033063e-08, 8.51381372e-07, 7.15888219e-07],
       [5.15144321e-06, 1.06571097e-04, 1.11536754e-04, 2.88029818e-04,
        4.13799617e-05, 6.17772913e-05, 4.58172484e-07, 1.53673483e-05,
        1.27477429e-05, 4.45914785e-07, 2.25230224e-07, 4.14239616e-10,
        2.37575568e-07, 3.03033063e-08, 8.51381372e-07, 7.15888219e-07]])

In [86]:
std_ctr_div_df = np.sum(np.square(np.mean(session_level_vals,axis=0) - session_level_vals)/df,axis=0)


In [87]:
std_ctr_div_df += np.sum(np.square(np.mean(session_level_vals,axis=0) - session_level_vals)/df,axis=0)

In [89]:
np.sqrt(std_ctr_div_df)

array([4.53935820e-03, 2.06466556e-02, 2.11221925e-02, 3.39428825e-02,
       1.28654517e-02, 1.57197063e-02, 1.35376879e-03, 7.84024191e-03,
       7.14079628e-03, 1.33553702e-03, 9.49168529e-04, 4.07057547e-05,
       9.74834484e-04, 3.48156897e-04, 1.84540659e-03, 1.69220356e-03])

In [94]:
m = 10


array([7.34669371, 4.06368647, 3.11568803, 1.72282665, 0.6360266 ,
       0.37628207, 0.17400195, 0.41692668, 1.55313722, 0.06341017,
       0.02665391, 0.00980697, 0.0233092 , 0.03741874, 0.04026706,
       0.47896009])